<a href="https://colab.research.google.com/github/kokolight/MetacriticProject/blob/main/DataScienceProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DataScience Project


## Creating the data set - Crawling 


### Main problem -
* Gaming companies develop games for years and cannot predict if their game will be sucessfull or not.

### Data -
Metacritic -

Description: Games ranking and reviews

Website: https://www.metacritic.com/

### Web scraping

### Data we scrape:
- Name - V
- Release Date - V
- Platforms count - V 
- Metascore - V
- Userscore - V
- Critic reviews - 
- Number of players
- Developer
- Publisher
- Maturaty Rating
- Genres
- Number of users ratings
- Number of critics ratings - V
- Number of awards
- Resolution

Possible:

- Number of positive/mixed/negative ratings
- Sound
- Online Playable (yes/no) 

In [38]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import threading
import re

In [60]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36'}
#url = "https://www.metacritic.com/game/xbox-360/call-of-duty-modern-warfare-2"
#url = "https://www.metacritic.com/game/xbox-one/avicii-invector"
#url = "https://www.metacritic.com/game/wii/super-mario-galaxy"
#url = "https://www.metacritic.com/game/gamecube/ssx-3"
url = "https://www.metacritic.com/game/dreamcast/nba-2k2"
game_req = requests.get(url,headers=headers)
soup = BeautifulSoup(game_req.text)
#soup.select('div.feature_userscore')[0].select('.metascore_w.user.large.game')[0].text
#soup.find("li", class_="summary_detail product_platforms").find_all("a")
#soup.select('li.publisher')[0].find("a").text.strip()
#len(soup.select("li.product_platforms")[0].find_all("a"))+1 if soup.select("li.product_platforms") else 1

# Critic reviews count
#soup.select("div.metascore_summary")[0].find("span", class_="count").find("a").find("span").text.strip()

# User reviews count
#re.search(r"\d+", soup.select("div.side_details")[0].find("span", class_="count").find("a").text).group(0)

#soup.select('li.developer')[0].find("a").text
len(soup.find_all("div", class_="ranking_title"))
#soup.find("div", class_="metascore_w user")

0

In [61]:
base_url = "https://www.metacritic.com"
browse_url = base_url + '/browse/games/score/metascore/all/all/filtered?'

month_dict = {'January':'1', 'February':'2', 'March':'3', 'April':'4', 'May': '5', 'June':'6', 'July':'7',
                    'August':'8', 'September':'9', 'October':'10', 'November':'11', 'December':'12'}

soup = BeautifulSoup('html.parser')

def metacritic_games_data(start_page, end_page):
  headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36'}

  names = []
  metascores = []
  userscores = []
  num_of_platforms = []
  release_dates = []
  hrefs = []
  platforms = []
  publishers = []
  num_of_critic_reviews = []
  num_of_user_reviews = []
  num_of_awards = []
  developers = []

  for i in range(start_page, end_page):
    print(f"i = {i}")
    page_url = browse_url+'page={0}'.format(i)
    req = requests.get(page_url, headers=headers)
    soup.append(bs4.BeautifulSoup(req.text))

  games_records = soup.find_all("td", class_="clamp-summary-wrap")

  for game in games_records:
    title_obj = game.find('a', class_="title")
    title =title_obj.text
    href = title_obj.get("href")
    meta_score = game.find('div', class_="metascore_w").text
    platform = game.find("span", class_="data").text.strip()
    date=game.find("div", class_="clamp-details").find_all("span")[2].text.replace(",", "")
    date_splitted = date.split(" ")
    release_date="{day}-{month}-{year}".format(day = date_splitted[1], month = month_dict[date_splitted[0]] , year = date_splitted[2])

    # add the values
    names.append(title)
    hrefs.append(href)
    metascores.append(meta_score)
    platforms.append(platform)
    release_dates.append(release_date)
    
    # in game section
    game_url = base_url+''+href
    game_req = requests.get(game_url, headers=headers)
    print(f"URL: {game_url}")
    game_soup = BeautifulSoup(game_req.text, 'html.parser')
    #user_score = game_soup.select('div.feature_userscore')[0].select('.metascore_w.user.large.game')[0].text
    publisher = game_soup.select('li.publisher')[0].find("a").text.strip()
    platforms_count = len(soup.select("li.product_platforms")[0].find_all("a"))+1 if soup.select("li.product_platforms") else 1
    critic_reviews_count = game_soup.select("div.metascore_summary")[0].find("span", class_="count").find("a").find("span").text.strip()
    user_reviews_count = re.search(r"\d+", game_soup.select("div.side_details")[0].find("span", class_="count").find("a").text).group(0)
    developer = game_soup.select('li.developer')[0].find("a").text
    awards_count = len(game_soup.find_all("div", class_="ranking_title"))
    # add the values per game
    #userscores.append(user_score)
    num_of_platforms.append(platforms_count)
    num_of_critic_reviews.append(critic_reviews_count)
    num_of_user_reviews.append(user_reviews_count)
    developers.append(developer)
    publishers.append(publisher)
    num_of_awards.append(awards_count)
  df = pd.DataFrame({'name': names, 'metascore': metascores, 'awards_count': num_of_awards, 'publisher': publishers, 'developer': developers
                     , 'user_reviews_count': num_of_user_reviews, 'critic_reviews_count': num_of_critic_reviews,
                     'release_date': release_dates, 'platform': platforms, 'platforms_count': num_of_platforms})
  df.to_csv("./project_data/video_games_ratings.csv",  mode='a')
  return df

df = metacritic_games_data(1,2)
df


i = 1
URL: https://www.metacritic.com/game/xbox/burnout-3-takedown
URL: https://www.metacritic.com/game/pc/diablo
URL: https://www.metacritic.com/game/playstation-2/metal-gear-solid-3-subsistence
URL: https://www.metacritic.com/game/xbox-360/call-of-duty-modern-warfare-2
URL: https://www.metacritic.com/game/playstation-3/metal-gear-solid-4-guns-of-the-patriots
URL: https://www.metacritic.com/game/playstation-2/ncaa-football-2004
URL: https://www.metacritic.com/game/playstation-2/god-of-war
URL: https://www.metacritic.com/game/xbox/star-wars-knights-of-the-old-republic
URL: https://www.metacritic.com/game/pc/sid-meiers-civilization-iv
URL: https://www.metacritic.com/game/gamecube/madden-nfl-2004
URL: https://www.metacritic.com/game/playstation-2/virtua-fighter-4
URL: https://www.metacritic.com/game/nintendo-64/super-mario-64
URL: https://www.metacritic.com/game/pc/the-witcher-3-wild-hunt
URL: https://www.metacritic.com/game/playstation-2/burnout-3-takedown
URL: https://www.metacritic.co

,name,metascore,awards_count,publisher,developer,user_reviews_count,critic_reviews_count,release_date,platform,platforms_count
0,Burnout 3: Takedown,94,3,EA Games,Criterion Games,184,76,7-9-2004,Xbox,1
1,Diablo,94,3,Blizzard Entertainment,Blizzard Entertainment,1208,12,31-12-1996,PC,1
2,Metal Gear Solid 3: Subsistence,94,3,Konami,Aspect,674,53,14-3-2006,PlayStation 2,1
3,Call of Duty: Modern Warfare 2,94,3,Activision,Infinity Ward,2984,100,10-11-2009,Xbox 360,1
4,Metal Gear Solid 4: Guns of the Patriots,94,3,Konami,Kojima Productions,4129,82,12-6-2008,PlayStation 3,1
...,...,...,...,...,...,...,...,...,...,...
95,Journey,92,3,SCEA,Tricky Pixels,2380,45,21-7-2015,PlayStation 4,1
96,Bloodborne,92,3,SCEA,From Software,11090,100,24-3-2015,PlayStation 4,1
97,Eternal Darkness: Sanity's Requiem,92,3,Nintendo,Silicon Knights,223,41,23-6-2002,GameCube,1
98,Warcraft III: Reign of Chaos,92,3,Blizzard Entertainment,Blizzard Entertainment,2111,40,3-7-2002,PC,1


In [ ]:
f.describe()